# Import statements

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
import random
import os,sys

# import keras
import keras
from keras import backend as K
backend_keras = keras.backend.backend()
print("keras is using", backend_keras, "as the backend")
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model, Sequential
from keras.models import load_model

from keras.optimizers import SGD, Nadam, Adamax, Adam, Adadelta, Adagrad, RMSprop


keras is using tensorflow as the backend


# Load redshift data and labels
These are the ones which have been created with the keras notebook

In [11]:
f = os.listdir()
f.sort()
for i in f: print(i)

.ipynb_checkpoints
CNN3_classify_redshift_with_autoencoder.ipynb
CNN4_classify_esc_frac_direct_cnn.ipynb
CNN5_classify_esc_frac_with_autoencoder.ipynb
CNN6_classify_redshift_direct_cnn.ipynb
big_mix_data_6528.npy
big_mix_labels_6528.npy
data_test.npy
data_train.npy
data_val.npy
figures
keras_arch_05.0_with_encoder_esc_predict.model
keras_arch_05.1_with_encoder_esc_predict.model
keras_arch_05.2_with_encoder_esc_predict.model
keras_arch_05.3_with_encoder_esc_predict.model
keras_arch_05.4.5_with_encoder_esc_predict.model
keras_arch_05.4.75_with_encoder_esc_predict.model
keras_arch_05.4_with_encoder_esc_predict.model
labels_test.npy
labels_train.npy
labels_val.npy
preprocessing.ipynb
trained models
training_perf_250_arch_03.0_pred_redshift_with_encoder.png


In [12]:
# load data
x_train_raw = np.load("data_train.npy",allow_pickle=True)
x_val_raw = np.load("data_val.npy")
x_test_raw = np.load("data_test.npy")

# load labels
labels_train = np.load("labels_train.npy",allow_pickle=True)
labels_val = np.load("labels_val.npy",allow_pickle=True)
labels_test = np.load("labels_test.npy",allow_pickle=True)

# we only care about the redshift labels here
lab_esc_train = np.array([i[0] for i in labels_train])
lab_esc_val = np.array([i[0] for i in labels_val])
lab_esc_test = np.array([i[0] for i in labels_test])

print("shapes :", x_train_raw.shape, x_val_raw.shape, x_test_raw.shape)
print("more shape :", lab_esc_train.shape, lab_esc_val.shape, lab_esc_test.shape)

shapes : (4896, 200, 200, 1) (652, 200, 200, 1) (980, 200, 200, 1)
more shape : (4896, 12) (652, 12) (980, 12)


In [13]:
x_train = x_train_raw
x_val = x_val_raw
x_test = x_test_raw

# Define the CNN architecture

In [23]:
classifier = Sequential()

# extract features
classifier.add(Conv2D(16, (3,3), input_shape=(200,200,1),
                     activation='relu', padding='valid'))
# reduce dimensionality, keep most important info
classifier.add(MaxPooling2D(pool_size=(3,3)))

# add a second layer, again with no padding
classifier.add(Conv2D(8, (3,3), activation='relu', padding='valid'))

# pool again
classifier.add(MaxPooling2D(pool_size=(3,3)))

# here i flatten and then guess the class
classifier.add(Flatten())

# fully connected layers ensures connections to all activations 
# in previous layers

classifier.add(Dense(units=12, activation='softmax'))

# Hyper params + compile network

In [24]:
# SGD, Nadam, Adamax, Adam, Adadelta, Adagrad, RMSprop

opt = RMSprop(learning_rate=0.001, rho=0.9)

classifier.compile(optimizer=opt,
                   loss='categorical_crossentropy',
                   metrics=["accuracy"])
print("printing summary of model")
classifier.summary()

printing summary of model
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 198, 198, 16)      160       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 66, 66, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 64, 64, 8)         1160      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 21, 21, 8)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3528)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 12)                42348     
Total params: 43,668
Trainable params: 43,668
Non-trainable params: 0
________________________

In [26]:
print(x_train.shape, lab_esc_train.shape)

(4896, 200, 200, 1) (4896, 12)


# Train the network

In [ ]:
EPOCHS = 50

H = classifier.fit(x_train, lab_esc_train,
                  epochs = EPOCHS,
                  batch_size = 128,
                  shuffle = True,
                  validation_data=(x_val, lab_esc_val))

Train on 4896 samples, validate on 652 samples
Epoch 1/50
4896/4896 [==============================] - 59s 12ms/step - loss: 2.4837 - accuracy: 0.0778 - val_loss: 2.4836 - val_accuracy: 0.0675
Epoch 2/50
4896/4896 [==============================] - 59s 12ms/step - loss: 2.4807 - accuracy: 0.0854 - val_loss: 2.4838 - val_accuracy: 0.0813
Epoch 3/50
4896/4896 [==============================] - 59s 12ms/step - loss: 2.4785 - accuracy: 0.0956 - val_loss: 2.4847 - val_accuracy: 0.0613
Epoch 4/50
4896/4896 [==============================] - 60s 12ms/step - loss: 2.4758 - accuracy: 0.1001 - val_loss: 2.4877 - val_accuracy: 0.0936
Epoch 5/50
4896/4896 [==============================] - 62s 13ms/step - loss: 2.4719 - accuracy: 0.1054 - val_loss: 2.4924 - val_accuracy: 0.0629
Epoch 6/50
4896/4896 [==============================] - 59s 12ms/step - loss: 2.4671 - accuracy: 0.1103 - val_loss: 2.5051 - val_accuracy: 0.0690
Epoch 7/50
4896/4896 [==============================] - 60s 12ms/step - loss:

In [ ]:
# plot the training loss and accuracy for each epoch
N = np.arange(0, EPOCHS)
plt.figure()
plt.style.use("ggplot")
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_accuracy")
plt.plot(N, H.history["val_accuracy"], label="val_accuracy")
plt.title("Predicting Redshift model 2")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("training_perf_arch_03.5_pred_esc_no_encoder.png")
plt.show()

# save the weights

In [ ]:
classifier.save("keras_arch_05.5_with_encoder_esc_predict.model")

# Display netowrk stats and save plots

# OR load trained CNN

# Make prediction